多线程类似于同时执行多个不同程序，多线程运行有如下优点：

使用线程可以把占据长时间的程序中的任务放到后台去处理。
用户界面可以更加吸引人，比如用户点击了一个按钮去触发某些事件的处理，可以弹出一个进度条来显示处理的进度
程序的运行速度可能加快
在一些等待的任务实现上如用户输入、文件读写和网络收发数据等，线程就比较有用了。在这种情况下我们可以释放一些珍贵的资源如内存占用等等。
线程在执行过程中与进程还是有区别的。每个独立的线程有一个程序运行的入口、顺序执行序列和程序的出口。但是线程不能够独立执行，必须依存在应用程序中，由应用程序提供多个线程执行控制。

每个线程都有他自己的一组CPU寄存器，称为线程的上下文，该上下文反映了线程上次运行该线程的CPU寄存器的状态。

指令指针和堆栈指针寄存器是线程上下文中两个最重要的寄存器，线程总是在进程得到上下文中运行的，这些地址都用于标志拥有线程的进程地址空间中的内存。

线程可以被抢占（中断）。
在其他线程正在运行时，线程可以暂时搁置（也称为睡眠） -- 这就是线程的退让。

In [3]:
#!/usr/bin/python3

import _thread
import time

# 为线程定义一个函数
def print_time( threadName, delay):
    count = 0
    while count < 5:
        time.sleep(delay)
        count += 1
        print ("%s: %s" % ( threadName, time.ctime(time.time()) ))

# 创建两个线程
try:
    _thread.start_new_thread( print_time, ("Thread-1", 2, ) )
    _thread.start_new_thread( print_time, ("Thread-2", 4, ) )
except:
    print ("Error: 无法启动线程")

while 1:
    pass

Thread-1: Fri May 10 14:41:10 2019
Thread-1: Fri May 10 14:41:12 2019
Thread-2: Fri May 10 14:41:12 2019
Thread-1: Fri May 10 14:41:14 2019
Thread-1: Fri May 10 14:41:16 2019
Thread-2: Fri May 10 14:41:16 2019
Thread-1: Fri May 10 14:41:18 2019
Thread-2: Fri May 10 14:41:20 2019
Thread-2: Fri May 10 14:41:24 2019
Thread-2: Fri May 10 14:41:28 2019


KeyboardInterrupt: 

两个线程分配到的cpu时间，按照打印的顺序输出出来了

In [9]:
import threading, time, random
count = 0
lock = threading.Lock()
def doAdd():
    '''@summary: 将全局变量count 逐一的增加10000。
    '''
    global count, lock
    lock.acquire()
    for i in range(10000):
        count = count + 1
        if count%1000==1:
            print(count)
    lock.release()
for i in range(2):
    threading.Thread(target = doAdd, args = (), name = 'thread-' + str(i)).start()
time.sleep(2)	#确保线程都执行完毕
# 保证线程顺序执行有什么用处了

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001


In [11]:
import threading, time
def doWaiting():
    print('start waiting:', time.strftime('%H:%M:%S'))
    time.sleep(3)
    print('stop waiting', time.strftime('%H:%M:%S'))
thread1 = threading.Thread(target = doWaiting)
thread1.start()
time.sleep(1)  #确保线程thread1已经启动
print('start join')
thread1.join()	#将一直堵塞，直到thread1运行结束。
print('end join')

start waiting: 17:11:53
start join
end join
stop waiting 17:11:56


In [14]:
#---- Condition
#---- 捉迷藏的游戏
import threading, time
class Hider(threading.Thread):
    def __init__(self, cond, name):
        super(Hider, self).__init__()
        self.cond = cond
        self.name = name
    
    def run(self):
        time.sleep(1) #确保先运行Seeker中的方法   
        
        self.cond.acquire() #b    
        print(self.name + ': 我已经把眼睛蒙上了')
        self.cond.notify()
        self.cond.wait() #c    
                         #f 
        print(self.name + ': 我找到你了 ~_~')
        self.cond.notify()
        self.cond.release()
                            #g
        print(self.name + ': 我赢了' )  #h
        
class Seeker(threading.Thread):
    def __init__(self, cond, name):
        super(Seeker, self).__init__()
        self.cond = cond
        self.name = name
    def run(self):
        self.cond.acquire()
        self.cond.wait()    #a    #释放对琐的占用，同时线程挂起在这里，直到被notify并重新占有琐。
                            #d
        print(self.name + ': 我已经藏好了，你快来找我吧')
        self.cond.notify()
        self.cond.wait()    #e
                            #h
        self.cond.release() 
        print(self.name + ': 被你找到了，哎~~~')
        
cond = threading.Condition()
seeker = Seeker(cond, 'seeker')
hider = Hider(cond, 'hider')
seeker.start()
hider.start()

hider: 我已经把眼睛蒙上了
seeker: 我已经藏好了，你快来找我吧
hider: 我找到你了 ~_~
hider: 我赢了
seeker: 被你找到了，哎~~~
